In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231/assignment1'
FOLDERNAME = 'Introduction to Speech Processing/'
assert FOLDERNAME is not None, "[!] Enter the folername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

%cd /content/drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Introduction to Speech Processing


In [2]:
!pip install jiwer
!pip install tokenizers
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.8 MB/s eta 0:00:00


In [3]:
import os

import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim

from librosa import effects

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from jiwer import wer, cer

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from torchaudio.models.decoder import ctc_decoder
from tokenizers.pre_tokenizers import Whitespace

from collections import defaultdict
from tensorboardX import SummaryWriter

VOCAB_SIZE = 198

/usr/local/lib/python3.10/dist-packages/torchaudio/models/decoder/_ctc_decoder.py:62: UserWarning: The built-in flashlight integration is deprecated, and will be removed in future release. Please install flashlight-text. https://pypi.org/project/flashlight-text/ For the detail of CTC decoder migration, please see https://github.com/pytorch/audio/issues/3088.
  warnings.warn(


In [4]:
class AudioTranscriptDataset(Dataset):
    """ Custom dataset to zip audio files and their matching transcripts """

    def __init__(self, audio_folder, transcript_folder, audio_extension=".wav", transcript_extension=".txt"):
        self.audio_folder = audio_folder
        self.transcript_folder = transcript_folder
        self.audio_extension = audio_extension
        self.transcript_extension = transcript_extension
        self.audio_files = os.listdir(audio_folder)

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = os.path.join(self.audio_folder, self.audio_files[idx])
        transcript_file = os.path.join(self.transcript_folder,
                                       self.audio_files[idx].replace(self.audio_extension, self.transcript_extension))
        waveform, sample_rate = torchaudio.load(audio_file)
        with open(transcript_file, "r") as f:
            transcript = f.read().strip()

        return waveform, transcript


In [5]:
class TextTransform:
    """Maps characters to integers and vice versa"""

    def __init__(self):
        # Initialize the tokenizer
        self.tokenizer = Tokenizer.from_file(rf"./tokenizer-train_{VOCAB_SIZE}.json")

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        text = text.replace(" ", "[SPACE]")
        return self.tokenizer.encode(text).ids

        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.tokenizer.token_to_id("[SPACE]")
            else:
                ch = self.tokenizer.token_to_id(c)
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.tokenizer.id_to_token(int(i)))
        return ''.join(string).replace('[SPACE]', ' ')

    def int_to_text_with_space(self, labels):
      """ Use a character map and convert integer labels to an text sequence """
      string = []
      for i in labels:
          string.append(self.tokenizer.id_to_token(int(i)))
          string.append(' ')
      return ''.join(string).replace('[SPACE]', ' ')


text_transform = TextTransform()

In [6]:
def GreedyDecoder(output, labels, label_lengths, blank_label=VOCAB_SIZE, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j - 1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets


In [7]:
# Define the CTC model
class CTCModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout=0.3, ckpt_path=None):
        super(CTCModel, self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, dropout=dropout, bidirectional=True,
                           batch_first=True)
        self.linear = nn.Linear(hidden_dim * 2, output_dim)  # *2 for bidirectional
        self.softmax = nn.LogSoftmax(dim=-1)

        # Initilize model's weigths when a path is given
        if ckpt_path is not None:
            self.load_state_dict(torch.load(ckpt_path))

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        linear_out = self.linear(rnn_out)
        return self.softmax(linear_out)

In [8]:
# Define the dataset and data loader
def load_an4_dataset(mode='train'):
    # Load the AN4 dataset and adjust the path accordingly
    audio_folder = f"./an4/{mode}/an4/wav/"
    transcript_folder = f"./an4/{mode}/an4/txt/"
    dataset = AudioTranscriptDataset(audio_folder, transcript_folder)

    return dataset


# MFCC feature extraction
# copied from fairseq/examples/hubert/simple_kmeans/dump_mfcc_feature.py on github
def get_feats(waveform):
    with torch.no_grad():
        x = waveform.float()
        x = x.view(1, -1)

        mfccs = torchaudio.compliance.kaldi.mfcc(
            waveform=x,
            sample_frequency=16000,
            use_energy=False,
        )  # (time, freq)
        mfccs = mfccs.transpose(0, 1)  # (freq, time)
        deltas = torchaudio.functional.compute_deltas(mfccs)
        ddeltas = torchaudio.functional.compute_deltas(deltas)
        concat = torch.cat([mfccs, deltas, ddeltas], dim=0)
        concat = concat.transpose(0, 1).contiguous()  # (freq, time)
        return concat


# Define the preprocessing function for MFCC feature extraction and target labels extraction
def data_processing(data, mode):
    mfccs = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, transcript) in data:
        # Augment the waveform with probability
        if mode == 'train':
            waveform = augment_waveform(waveform)
        # Use 39 MFCC coefficients
        mfcc = get_feats(waveform)
        mfccs.append(mfcc)
        label = torch.Tensor(text_transform.text_to_int(transcript))
        labels.append(label)
        input_lengths.append(mfcc.shape[0])
        label_lengths.append(len(label))

    mfccs = nn.utils.rnn.pad_sequence(mfccs, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=1)
    return mfccs, labels, torch.tensor([mfccs.shape[1]] * mfccs.shape[0]), torch.tensor(
        [labels.shape[1]] * labels.shape[0])  # input_lengths, label_lengths


def augment_waveform(waveform, augmentation_probability=0.7, noise_level=0.02):
    """
    Augment a waveform by adding random noise with the given probability.
    """
    if torch.rand(1) < augmentation_probability:
        # Add random noise to the waveform
        noise = torch.randn_like(waveform) * noise_level
        augmented_waveform = waveform + noise
        return augmented_waveform

    return waveform

In [9]:
# Training function
def train(model, batch_size, print_interval, save_ckpt_interval, criterion, optimizer, device):
    # Load the AN4 train dataset
    train_dataset = load_an4_dataset(mode='train')

    # Preprocess the dataset and create the data loader
    data_loader = DataLoader(dataset=train_dataset,
                             batch_size=batch_size,
                             shuffle=True,
                             collate_fn=lambda x: data_processing(x, mode='train'))
    model.train()
    total_loss = 0.0

    for batch_idx, _data in enumerate(data_loader):
        batch_inputs, targets, input_lengths, target_lengths = _data
        batch_inputs, targets = batch_inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # Get model predictions
        outputs = model(batch_inputs).transpose(0, 1)

        # Calculate the CTC loss
        loss = criterion(outputs, targets, input_lengths, target_lengths)

        # Backpropagation and optimization step
        loss.backward()
        optimizer.step()

        # Print current batch loss
        if batch_idx % print_interval == 0:
            print(f"Batch {batch_idx}, Loss: {loss:.4f}")

        # Save the trained model
        # if batch_idx % save_ckpt_interval == 1:
        #   torch.save(model.state_dict(), "ctc_model.pt")

        total_loss += loss.item()

    return total_loss / len(data_loader)

In [10]:
def test(model, device, batch_size, criterion, mode='test', std_output=False):
    # Load the AN4 test dataset
    test_dataset = load_an4_dataset(mode=mode)

    # Preprocess the dataset and create the data loader
    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             collate_fn=lambda x: data_processing(x, mode='test'))

    if std_output:
      print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            mfccs, labels, input_lengths, label_lengths = _data
            mfccs, labels = mfccs.to(device), labels.to(device)

            output = model(mfccs)  # (batch, time, n_class)
            output = output.transpose(0, 1)  # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            # decoded_preds = tokenizer.decode_batch(torch.argmax(output.transpose(0, 1), dim=2).tolist())
            # labels = labels.tolist()
            # labels = [list(map(int, label.tolist())) for label in labels]
            # decoded_targets = tokenizer.decode_batch(labels)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)

            for j in range(len(decoded_preds)):
                if std_output:
                  print("Target: " + decoded_targets[j])
                  print("Predicted: " + decoded_preds[j])
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))

    avg_cer = sum(test_cer) / len(test_cer)
    avg_wer = sum(test_wer) / len(test_wer)

    if std_output:
      print('Test set: Average loss: {:.4f}, Average CER: {:4f}% Average WER: {:.4f}%\n'.format(test_loss, avg_cer * 100,
                                                                                                avg_wer * 100))
    return avg_cer, avg_wer, test_loss

In [11]:
def generate_lexicon():
    # tokenizer = text_transform.tokenizer
    dataset = load_an4_dataset(mode='train')
    lexicon = defaultdict(list)

    for example in dataset:
        words = example[1].split()  # Split transcription into words

        for word in words:
            tokens = text_transform.text_to_int(word)
            sub_words = text_transform.int_to_text_with_space(tokens)
            if sub_words+"|" not in lexicon[word]:
              lexicon[word].append(sub_words+ "|")

    # Save the lexicon to a file
    lexicon_path = "lexicon.txt"
    with open(lexicon_path, 'w') as f:
        for word, tokens_list in lexicon.items():
            f.write(f"{word} {' '.join(tokens_list)}\n")

    print("Lexicon generated and saved to", lexicon_path)

In [12]:
def loss_on_training_and_val_graph(input_dim, hidden_dim, output_dim, batch_size, print_interval,
                                   save_ckpt_interval, learning_rate, regularization, device):
    # Initialize SummaryWriter
    log_dir = "logs_loss"  # Directory to store the logs
    writer = SummaryWriter(log_dir=log_dir)

    # Initialize ASR model with the current hidden_dim
    model = CTCModel(input_dim, hidden_dim, output_dim).to(device)

    # Loss function and optimizer
    criterion = CTCLoss(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=regularization)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

    num_epochs = 150
    for epoch in range(num_epochs):

        avg_loss = train(model, batch_size, print_interval, save_ckpt_interval, criterion, optimizer, device)
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f} Hidden Dim: {hidden_dim}")
        scheduler.step()

        if epoch % 5 == 0:

          train_CER, train_WER, train_set_loss = test(model, device, batch_size, criterion, 'train', std_output=False)
          writer.add_scalars('Training Set Loss', {f'hidden_dim_{hidden_dim}':avg_loss}, epoch)
          writer.add_scalars('Training Set CER %', {f'hidden_dim_{hidden_dim}':train_CER}, epoch)
          writer.add_scalars('Training Set WER %', {f'hidden_dim_{hidden_dim}':train_WER}, epoch)

          val_CER, val_WER, val_set_loss = test(model, device, batch_size, criterion, 'val', std_output=False)
          writer.add_scalars('Validation Set Loss', {f'hidden_dim_{hidden_dim}':val_set_loss}, epoch)
          writer.add_scalars('Validation Set CER %', {f'hidden_dim_{hidden_dim}':val_CER}, epoch)
          writer.add_scalars('Validation Set WER %', {f'hidden_dim_{hidden_dim}':val_WER}, epoch)

          test_CER, test_WER, test_set_loss = test(model, device, batch_size, criterion, 'test', std_output=False)
          writer.add_scalars('test Set Loss', {f'hidden_dim_{hidden_dim}':test_set_loss}, epoch)
          writer.add_scalars('test Set CER %', {f'hidden_dim_{hidden_dim}':test_CER}, epoch)
          writer.add_scalars('test Set WER %', {f'hidden_dim_{hidden_dim}':test_WER}, epoch)

    # Close the SummaryWriter
    writer.close()
    return model

In [13]:
# CTC loss function
class CTCLoss(nn.Module):
    def __init__(self, device):
        super(CTCLoss, self).__init__()
        self.ctc_loss = nn.CTCLoss(blank=VOCAB_SIZE).to(device)

    def forward(self, log_probs, targets, input_lengths, target_lengths):
        return self.ctc_loss(log_probs, targets, input_lengths, target_lengths)

In [14]:
# Hyperparameters
input_dim = 39
hidden_dim = 512
output_dim = VOCAB_SIZE + 1  # Number of characters in AN4 dataset, including blank (249)

batch_size = 16
print_interval = 15
save_ckpt_interval = 45
learning_rate = 0.0005
regularization = 0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = CTCLoss(device)

model = loss_on_training_and_val_graph(input_dim, hidden_dim, output_dim, batch_size, print_interval,
                                       save_ckpt_interval, learning_rate, regularization, device)
# save final weights
torch.save(model.state_dict(), "./sub_token_ctc_lstm_model.pt")

Batch 0, Loss: 122.8884
Batch 15, Loss: 4.4364
Batch 30, Loss: 2.8220
Batch 45, Loss: 2.4059
Epoch 1/150, Average Loss: 14.7175 Hidden Dim: 512
Batch 0, Loss: 1.7422
Batch 15, Loss: 1.7895
Batch 30, Loss: 1.5720
Batch 45, Loss: 1.8137
Epoch 2/150, Average Loss: 1.7352 Hidden Dim: 512
Batch 0, Loss: 1.5138
Batch 15, Loss: 1.3695
Batch 30, Loss: 1.7325
Batch 45, Loss: 1.2940
Epoch 3/150, Average Loss: 1.5042 Hidden Dim: 512
Batch 0, Loss: 1.5129
Batch 15, Loss: 1.1339
Batch 30, Loss: 1.2350
Batch 45, Loss: 1.5071
Epoch 4/150, Average Loss: 1.4341 Hidden Dim: 512
Batch 0, Loss: 1.3741
Batch 15, Loss: 1.3891
Batch 30, Loss: 1.4604
Batch 45, Loss: 1.4023
Epoch 5/150, Average Loss: 1.4214 Hidden Dim: 512
Batch 0, Loss: 1.2656
Batch 15, Loss: 1.4760
Batch 30, Loss: 1.4040
Batch 45, Loss: 1.3723
Epoch 6/150, Average Loss: 1.3936 Hidden Dim: 512
Batch 0, Loss: 1.2786
Batch 15, Loss: 1.4711
Batch 30, Loss: 1.3823
Batch 45, Loss: 1.5842
Epoch 7/150, Average Loss: 1.3941 Hidden Dim: 512
Batch 0, L

In [15]:
model = CTCModel(input_dim, hidden_dim, output_dim, ckpt_path="./sub_token_ctc_lstm_model.pt").to(device)
test(model, device, batch_size, criterion, mode='train', std_output=True)


evaluating...
Target: HELP                      
Predicted: HELP  
Target: FIVE TWO SEVEN                  
Predicted: FIVE TWO SEVEN  
Target: O A K D A L E D R I V E
Predicted: O A K D A L E D R I V 
Target: NO                      
Predicted: NO  
Target: ONE TWO FOUR ONE                
Predicted: ONE TWO FOUR ONE  
Target: M I C H A E L          
Predicted: M I C H A E L  
Target: B R O S T              
Predicted: B R O S T  
Target: RUBOUT F M Q N H THREE SEVENTY        
Predicted: RUBOUT F M Q N H THREE SEVENTY 
Target: GO                      
Predicted: GO  
Target: ONE FIVE TWO ZERO SEVEN              
Predicted: ONE FIVE TWO ZERO SEVEN  
Target: P I T T S B U R G H    
Predicted: P I T T S B U R G H
Target: L O O F B O U R R O W  
Predicted: L O O F B O U R R O  
Target: T V A H FIFTY TWO FIFTY THREE        
Predicted: T V A H FIFTY TWO FIFTY THREE 
Target: H I N I C H            
Predicted: H I N I C H  
Target: P I T T S B U R G H    
Predicted: P I T T S B U R G H  
Tar

(0.007117945531269119, 0.006577325891511117, 0.15358104708570017)

In [16]:
test(model, device, batch_size, criterion, mode='val', std_output=True)


evaluating...
Target: P I T T S B U R G H          
Predicted: P I T T S B U R G H H
Target: ERASE O T H F I FIVE ZERO              
Predicted: ERASE O T H F I FIVE ZERO  
Target: YES                            
Predicted: YES  
Target: EIGHT OH THREE TWO THREE FOUR FIVE NINE SEVEN TWO          
Predicted: EIGHT THREE TWO THREE FOUR FIVE NINE SEVEN TWO  
Target: S H A R O N                  
Predicted: S H A R O E  
Target: ONE SIXTEEN FORTY EIGHT                      
Predicted: ONE SIXTY FORTY EIGHT  
Target: FOUR ONE TWO FOUR TWO TWO NINE EIGHT TWO EIGHT          
Predicted: FOUR ONE TWO FOUR TWO TWO NINE EIGHT TWO  
Target: RUBOUT S H K J FIVE SEVEN SIX              
Predicted: RUBOUT S H H J FIVE SEVEN SIX  
Target: SIX EIGHT THREE ONE FIVE FIVE EIGHT                
Predicted: SIX EIGHT THREE ONE FIVE TWENTY EIGHT  
Target: SEPTEMBER FIRST NINETEEN SIXTY NINE                    
Predicted: SEPTEMBER NINETEEN SIXTY NINE  
Target: ONE TWO TWO TWO                      
Predicted: O

(0.13116003486777517, 0.15354026090868195, 0.3335225731134415)

In [17]:
test(model, device, batch_size, criterion, std_output=True)


evaluating...
Target: ONE FIVE TWO TWO SEVEN          
Predicted: FOUR FIRST TWO TWO SEVEN  
Target: ERASE K M H N I SIX OH FIVE  
Predicted: ERASE K M H M I SIX SEVEN FIVE  
Target: NO                  
Predicted: NO  
Target: ENTER EIGHT THIRTEEN              
Predicted: ENTER EIGHT FIFTY  
Target: ENTER ONE SEVENTY SIX            
Predicted: ENTER ONE SEVENTY SIX  
Target: ONE FIVE TWO ONE THREE          
Predicted: ONE FIVE TWO ONE THREE  
Target: M Y E R S          
Predicted: M Y E RI S  
Target: MAY SECOND NINETEEN SIXTY FIVE          
Predicted: THREE SECOND NINETEEN SIXTY FIVE  
Target: P H I N N E Y      
Predicted: P H I N N E Y 
Target: SIXTY SIX THIRTY THREE            
Predicted: SIX SIX THIRTY THREE  
Target: ENTER TWO NINE EIGHT ONE          
Predicted: ENTER TWO NINE EIGHT ONE  
Target: M E L V I N        
Predicted: M E L V I N 
Target: P I T T S B U R G H
Predicted: P I T T S B G H
Target: C E D A R V I L L E
Predicted: C E D A V I L L 
Target: L E V I S O N      
P

(0.12989920171206226, 0.16373016940324628, 0.3288292917940352)